<a href="https://colab.research.google.com/github/siting1206/NLP_HW2/blob/main/NLP_assignment2_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install pandas nltk
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 11.2 MB/s 
     |████████████████████████████████| 7.6 MB 47.3 MB/s 
     |████████████████████████████████| 182 kB 56.3 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch, pandas as pd
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
test_df = pd.read_csv('/content/drive/MyDrive/NLP_assignment2/Dataset/test_ret.txt', sep='\t')
test_df[['snippet','question']] = test_df[['snippet','question']].apply(lambda x: x.str.strip('\"'))

In [6]:
test_df.head()

,snippet,question
0,uk map geography united kingdom map united kin...,'s largest kingdom united kingdom
1,"hannah montana daily texan oct 7 , 2013 20 yea...",party u singer also plays young lady named han...
2,"peach tree fruit britannica com mar 4 , 2015 p...","part peach downy fuzzy , fruit 's called peach..."
3,4 x 12 thassos p akdo polished 4 x 12 thassos ...,4 x 12
4,written sound onomatopoeia dictionary word der...,verb bouncing basketball sounds like 're slobb...


## Tokenizer

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [8]:
test_df[['snippet','question']] = test_df[['snippet','question']].astype(str)

test_data_s = test_df['snippet'].tolist()
test_data_q = test_df['question'].tolist()

test_encodings = tokenizer(test_data_s, test_data_q, truncation=True, padding=True)
tokenizer.decode(test_encodings['input_ids'][0])

"[CLS] uk map geography united kingdom map united kingdom printable map united kingdom info links united kingdom facts, famous's fifth largest economy world, second largest europe, united kingdom country profile united kingdom great britain united kingdom facts, united kingdom geography, travel united kingdom, united kingdom zenith, british empire stretched one fourth earth's surface uk's second largest airline, serves 30 european destinations [SEP]'s largest kingdom united kingdom [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

## Dataset

In [9]:
class qrDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [10]:
test_dataset = qrDataset(test_encodings)

In [11]:
next(iter(test_dataset))

{'input_ids': tensor([  101, 26006,  4520, 14534, 10280,  6139,  4520, 10280,  6139,  5911,
          1895,  4520, 10280,  6139, 23992,  6743, 10280,  6139,  9193,   117,
          2505,   112,   188,  3049,  2026,  4190,  1362,   117,  1248,  2026,
         27772,  3186,   117, 10280,  6139,  1583,  6168, 10280,  6139,  1632,
          9304,  5168,  1394, 10280,  6139,  9193,   117, 10280,  6139, 14534,
           117,  3201, 10280,  6139,   117, 10280,  6139,   195, 21462,  1582,
           117,  9304, 10721,  1324,  8207,  6572,  1141,  2223,  4033,   112,
           188,  2473, 26006,   112,   188,  1248,  2026,  8694,   117,  3411,
          1476, 27772,  3186,  1389, 15309,   102,   112,   188,  2026,  6139,
         10280,  6139,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

## Model

In [12]:
from transformers import BertModel

class myModel(torch.nn.Module):

    def __init__(self):

        super(myModel, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.fc = nn.Linear(768, 2)
        

    def forward(self, input_ids, attention_mask, token_type_ids):

        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=True)
        logits = output[0]
        out = self.fc(logits)

        return out

In [13]:
from transformers import AdamW
from tqdm import tqdm

# Set GPU / CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Put model on device
model = myModel().to(device)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
batch_size = 8
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [15]:
model = myModel().to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/NLP_assignment2/bert_model'))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

## Predict

In [16]:
def predict(test_loader):
    predict_pos = []

    model.eval()

    ans_sub_output = []

    loop = tqdm(test_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)

        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        ans_start_logits, ans_end_logits = torch.split(outputs, 1, 2)

        ans_start_logits = ans_start_logits.squeeze(-1).contiguous()
        ans_end_logits = ans_end_logits.squeeze(-1).contiguous()

        ans_start_prdict = torch.argmax(ans_start_logits, 1).cpu().numpy()
        ans_end_prdict = torch.argmax(ans_end_logits, 1).cpu().numpy()

        for i in range(len(input_ids)):
            predict_pos.append((ans_start_prdict[i].item(), ans_end_prdict[i].item()))

            ans_sub = tokenizer.decode(input_ids[i][ans_start_prdict[i]:ans_end_prdict[i]+1])
            
            ans_sub_output.append(ans_sub)
    
    return ans_sub_output, predict_pos

In [17]:
ans_sub_output, predict_pos = predict(test_loader)

100%|██████████| 3406/3406 [08:26<00:00,  6.73it/s]


In [18]:
def get_output_post_fn(test, ans_sub_output):
    ans_sub = []
    for i in range(len(test)):

        ans_sub_pred = ans_sub_output[i].split()

        if ans_sub_pred is None:
            ans_sub_pred = []
        ans_sub_error_index = ans_sub_pred.index('[SEP]') if '[SEP]' in ans_sub_pred else -1

        if ans_sub_error_index != -1:
            ans_sub_pred = ans_sub_pred[:ans_sub_error_index]


        ans_sub.append(' '.join(ans_sub_pred))

    return ans_sub

In [19]:
ans_sub = get_output_post_fn(test_df, ans_sub_output)

In [20]:
test_df['ans_sub'] = ans_sub

In [21]:
test_df.head()

,snippet,question,ans_sub
0,uk map geography united kingdom map united kin...,'s largest kingdom united kingdom,uk
1,"hannah montana daily texan oct 7 , 2013 20 yea...",party u singer also plays young lady named han...,montana
2,"peach tree fruit britannica com mar 4 , 2015 p...","part peach downy fuzzy , fruit 's called peach...",
3,4 x 12 thassos p akdo polished 4 x 12 thassos ...,4 x 12,calacatta
4,written sound onomatopoeia dictionary word der...,verb bouncing basketball sounds like 're slobb...,bee


## Output

In [22]:
test = test_df.drop(['snippet'], axis=1)
test.head()

,question,ans_sub
0,'s largest kingdom united kingdom,uk
1,party u singer also plays young lady named han...,montana
2,"part peach downy fuzzy , fruit 's called peach...",
3,4 x 12,calacatta
4,verb bouncing basketball sounds like 're slobb...,bee


In [40]:
f = open("/content/drive/MyDrive/NLP_assignment2/Dataset/test-submit.txt", "w")
for i in range(len(test)):
  f.write(test['question'][i] + " ||| " + test['ans_sub'][i])
  f.write("\n")
f.close()